In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.utils import shuffle
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import os 

In [2]:
import numpy as np
import h5py
from tqdm import tqdm

from PIL import Image
import torch

In [3]:
print(np.__version__)
print(pd.__version__)
print(rdkit.__version__)

1.19.1
1.1.1
2020.03.3


In [4]:
smiles_list = list()

with open('CID-SMILES', 'r') as smiles:
    extra_data = smiles.read()
extra_data = extra_data.split('\n')
extra_data = shuffle(extra_data)
#shuffled extra_data

for line in tqdm(extra_data[:1000000]):
    image_id, smiles = line.split('\t')
    m = Chem.MolFromSmiles(smiles)
    if m != None:    
        smiles_list.append(smiles) #shuffled extra_data in smiles_list
        
smiles_list = np.array(smiles_list) #convert list to np.array

  0%|          | 0/1000000 [00:00<?, ?it/s]RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
  0%|          | 865/1000000 [00:00<01:55, 8645.95it/s]RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
  0%|          | 3356/1000000 [00:00<02:00, 8302.20it/s]RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
  0%|          | 4975/1000000 [00:00<02:03, 8080.38it/s]RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:25:56] WARNING: not removing hydrogen atom without 

In [5]:
smiles_list.shape

(999325,)

In [6]:
with open('train.csv', 'r') as train_smiles:
    data = train_smiles.read()

train_list = list()
    
for line in tqdm(data.split('\n')[1:-1]):
    image_id, smiles = line.split(',')
    train_list.append(smiles)
    
train_list = np.array(train_list)

100%|██████████| 908765/908765 [00:00<00:00, 2267926.88it/s]


In [7]:
train_list = np.union1d(train_list, smiles_list) #train_list + smiles_list

In [8]:
train_list = shuffle(train_list)
train_list

array(['CS(=CC1=CC(=C(C=C1)F)F)(=O)O',
       'c1ccc(CN2CCN(Cc3ccccc3)C2c2cccs2)cc1',
       'OC(CC(CC(O)(C(F)(F)Cl)C(F)(F)Cl)=NNc1ccccc1)(C(F)(F)Cl)C(F)(F)Cl',
       ..., 'CC(C)CNC(=O)Nc1cc(=O)n(C)c(=O)n1C',
       'Cc1c(OC(=O)C2CCCCC2)ccc2c3c(c(=O)oc12)CCCC3',
       'CCCCOc1c(OC)cccc1C(=O)OCCN(C)C.CI'], dtype='<U1341')

In [9]:
smiles_dict = {'SMILES':[]}
for smiles in tqdm(enumerate(train_list)):
    smiles = smiles[1]
    smiles_dict['SMILES'].append(smiles)

1907741it [00:05, 340001.32it/s]


In [10]:
train = pd.DataFrame(smiles_dict)

In [20]:
train = train[train['SMILES'].str.len() <= 70]
train.index = range(train.shape[0])

In [21]:
train['file_name'] = 'train_' + train.index.astype('str') + '.png'
train = train[['file_name', 'SMILES']]
train

/home/toc/anaconda3/envs/dacon/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,file_name,SMILES
0,train_0.png,CS(=CC1=CC(=C(C=C1)F)F)(=O)O
1,train_1.png,c1ccc(CN2CCN(Cc3ccccc3)C2c2cccs2)cc1
2,train_2.png,OC(CC(CC(O)(C(F)(F)Cl)C(F)(F)Cl)=NNc1ccccc1)(C...
3,train_3.png,C=CCOC1=CC=CC(=C1)C2=NN3C=CC4=CC=CC=C4C3=C2
4,train_4.png,C1=CC(=C(C=C1C=CC(=O)NC2=NC3=C(O2)C=CC(=C3)Cl)...
...,...,...
1753731,train_1753731.png,CCOC(=O)c1[nH]c2ccc(OC)cc2c1NC(=O)CN1CCOCC1
1753732,train_1753732.png,COC1=CC=CC=C1N2CCN(CC2)C(=O)[C@H](CO)NC(=O)C(=...
1753733,train_1753733.png,CC(C)CNC(=O)Nc1cc(=O)n(C)c(=O)n1C
1753734,train_1753734.png,Cc1c(OC(=O)C2CCCCC2)ccc2c3c(c(=O)oc12)CCCC3


In [24]:
if not (os.path.isdir('extra_train(2)')): #train dir이 없다면 train dir 생성
    os.makedirs(os.path.join('extra_train(2)'))

In [25]:
path = './extra_train(2)/'

In [26]:
for idx, row in tqdm(train.iterrows()):
    file = row['file_name']
    smiles = row['SMILES']
    m = Chem.MolFromSmiles(smiles)
    if m != None:
        img = Draw.MolToImage(m, size=(300,300))
        img.save(path+file)

96it [00:00, 153.66it/s]RDKit WARNING: [00:03:59] WARNING: not removing hydrogen atom without neighbors
881it [00:05, 154.10it/s]RDKit WARNING: [00:04:04] WARNING: not removing hydrogen atom without neighbors
3233it [00:20, 157.81it/s]RDKit WARNING: [00:04:19] WARNING: not removing hydrogen atom without neighbors
3249it [00:20, 157.79it/s]RDKit WARNING: [00:04:19] WARNING: not removing hydrogen atom without neighbors
4134it [00:26, 154.28it/s]RDKit WARNING: [00:04:25] WARNING: not removing hydrogen atom without neighbors
7008it [00:45, 151.94it/s]RDKit WARNING: [00:04:43] WARNING: not removing hydrogen atom without neighbors
8470it [00:54, 155.94it/s]RDKit WARNING: [00:04:53] WARNING: not removing hydrogen atom without neighbors
10640it [01:08, 153.64it/s]RDKit WARNING: [00:05:07] WARNING: not removing hydrogen atom without neighbors
11717it [01:15, 157.09it/s]RDKit WARNING: [00:05:14] WARNING: not removing hydrogen atom without neighbors
11878it [01:16, 148.52it/s]RDKit WARNING: [00:0

In [27]:
import csv

In [28]:
with open('./extra_train(2).csv', 'w', newline='') as extra_train:
    datawriter = csv.writer(extra_train)
    
    datawriter.writerow(['file_name', 'SMILES'])
    for idx, row in tqdm(train.iterrows()):
        file = row['file_name']
        smiles = row['SMILES']
        datawriter.writerow([file, smiles])

1753736it [01:34, 18624.31it/s]


In [ ]:
f = h5py.File('./backup/extra_train(2)_images.hdf5')
dt = h5py.special_dtype(vlen=np.dtype('uint8'))
dset = f.create_dataset('images', (1907741, ), dtype=dt)

data_folder = './'
data_name = 'extra_train(2).csv'

with open(data_folder + data_name, 'r') as csv_file:
    data = csv_file.read()

index = 0

for line in tqdm(data.split('\n')[1:-1]):
    image_id, smiles = line.split(',')
    caption = "<" + smiles + ">"
    filename = data_folder + "extra_train(2)/"+ image_id
    #fin = open(filename, 'rb')
    #binary_data = fin.read()
    dset[index] = np.array(Image.open(filename).resize((224,224))).flatten()
    index += 1
    
    if index % 10000 == 0:
        print(index)

/home/toc/anaconda3/envs/dacon/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.
  1%|          | 10084/1753736 [00:17<50:42, 573.10it/s]

10000


  1%|          | 20108/1753736 [00:35<51:08, 564.98it/s]

20000


  2%|▏         | 30081/1753736 [00:52<50:05, 573.52it/s]

...

 63%|██████▎   | 1110062/1753736 [32:36<18:49, 569.77it/s]

1110000


 64%|██████▍   | 1120078/1753736 [32:54<18:30, 570.73it/s]

1120000


 64%|██████▍   | 1128307/1753736 [33:08<18:15, 570.79it/s]